In the last notebook, I wrote some code for running single column model simulations based on xarray data. Now, let's use it to dicuss the model spin-up error.

In [ ]:
%matplotlib inline

In [ ]:
import attr
from uwnet.columns import single_column_simulation
import torch

In [ ]:
ds = xr.open_dataset("../data/processed/training.nc")
model = torch.load("../models/113/3.pkl")

# The Spin Up problem

Let's run a single column simulation for a given location:

In [ ]:
location = ds.isel(x=slice(0,1), y=slice(32,33))
scm_data = single_column_simulation(model, location, interval=(0, 190))

In [ ]:
scm_data.QT.squeeze().T.plot.contourf(vmin=0)

All the very dark points are neagative. Indicating a large error.

Let's merge back in the SST and SOLIN field

In [ ]:
merged_pred_data = location.rename({'SLI': 'SLIOBS', 'QT': 'QTOBS'}).merge(scm_data, join='inner')

And compute the apparent source:

In [ ]:
apparent_source = model.call_with_xr(merged_pred_data).squeeze()
apparent_source.QT.T.plot()
plt.title("FQTNN during a single column simulation")

And the apparent source without allow the scheme to spin up looks like this:

In [ ]:
apparent_source = model.call_with_xr(location.isel(time=slice(0, 190))).squeeze()
apparent_source.QT.T.plot()
plt.title("FQTNN no spinup")

We can see there are some big differenences between these indicating the model is not spun up correctly.

## The short time spin up process

In [ ]:
merged_pred_data.QT[0].plot(y='z', label='Prediction after one step')
merged_pred_data.QTOBS[0].plot(y='z', label='OBS')
plt.legend()

In [ ]:
abs_error = merged_pred_data.QT[0] - merged_pred_data.QTOBS[0]
rel_error = np.abs(abs_error)/merged_pred_data.QTOBS[0]


plt.semilogx(rel_error.squeeze(), rel_error.z)
plt.xlabel("QT relative error")

The biggest percent errors in humidity are clearly in the upper troposphere. where the humidity is very small. Although the errors are not good lower down either.

## Is the model fully spun up after one time step?

In [ ]:
dt = 3 * 3600.0

def step(apparent_source, location, dt):
    # compute the total source
    total_source = apparent_source/86400 + location[['FSLI', 'FQT']].rename({'FQT': 'QT', 'FSLI': 'SLI'})
    # make a time step
    one_step = location + total_source * dt
    return one_step.shift(time=1)
    

one_step = step(apparent_source, location, dt)
# merge back in the SST and SOLIN
merged_one_step_data = location.rename({'SLI': 'SLIOBS', 'QT': 'QTOBS'}).merge(one_step, join='inner')
# compute the apparent source
one_step_apparent_source = model.call_with_xr(merged_one_step_data)
# plot
one_step_apparent_source.QT.plot(y='z')

this is not identical to the Q1 for the full single column run, but it is much closer. Thus it seems that the scheme is mostly spun up within one time step.

## Which variable is the spin up important for?

### Use only SLI from one step prediction

In [ ]:
merged_data_with_new_sli = location.rename({'SLI': 'SLIOBS'}).merge(one_step.drop('QT'), join='inner')
# compute the apparent source
one_step_apparent_source = model.call_with_xr(merged_data_with_new_sli)
# plot
one_step_apparent_source.QT.plot(y='z')

### Use only QT from one step prediction

In [ ]:
# merge back in the SST and SOLIN
merged_data_with_new_qt = location.rename({'QT': 'QTOBS'}).merge(one_step.drop('SLI'), join='inner')
# compute the apparent source
one_step_apparent_source = model.call_with_xr(merged_data_with_new_qt)
# plot
one_step_apparent_source.QT.plot(y='z')

### Conclusion

Clearly using the spun-up QT has a much more positive impact than using the spun-up SLI.

## Which levels of QT are most important?

In [ ]:
def merge_qt_levels_from_one_step(location, one_step, levels):
    out = xr.merge([location.QT, one_step.QT.rename('QTP')], join='inner')
    out.QT.values[:, levels] = out.QTP.values[:, levels]
    qt = out.QT
    return location.assign(QT=qt, SLI=one_step.SLI).dropna('time')


def get_apparent_source_from_mixed(location, one_step, levels):
    """Get apparent source merging by mixing the humidity field from location and one_step
    
    the specified levels of humidity are taken from the one step prediction, the rest are from the observed dataset (location).
    
    This should tell us which levels matter the most
    """
    mixed_qt_input = merge_qt_levels_from_one_step(location, one_step, levels)
    return model.call_with_xr(mixed_qt_input)

Using the spun-up variables for only the first 10 vertical levels does little

In [ ]:
lower_atmos = np.r_[:10]
get_apparent_source_from_mixed(location, one_step, lower_atmos).QT.T.plot()

Here we use the spun up variables for every point below the gray line.

In [ ]:
n = 30
m= 0
upper_atmos = np.r_[m:n]
get_apparent_source_from_mixed(location, one_step, upper_atmos).QT.T.plot(vmax=40)
plt.axhline(location.z[n], c='k', alpha=.4)
plt.axhline(location.z[m], c='k', alpha=.4)

We see that using the "spun up" moisutre has the moist drastic impact where humidity is sometimes negative. On the other hand, it is also somewhat important to spin up the boundary layer properly. Here, we use the spun up moisture only between the lines:

In [ ]:
n = 30
m= 10
upper_atmos = np.r_[m:n]
get_apparent_source_from_mixed(location, one_step, upper_atmos).QT.T.plot(vmax=40)
plt.axhline(location.z[n], c='k', alpha=.4)
plt.axhline(location.z[m], c='k', alpha=.4)

There are some small differences between these plots. indicating that the lower atmospheric points matter. They probably matter because of the extremely large SGS forcing in that layer.